In [ ]:
pip install wilds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=e707c2723c5e65a385874684678d5dd0156e2b9e85f85320b7b5fadfdfddf973
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-s5dqgjvf
  Running command git clone --filter=blob:none --quiet https://github.com/py

In [ ]:
from torchtext.vocab import GloVe
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import os
import pandas as pd
import numpy as np
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
import re
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
from wilds.common.grouper import CombinatorialGrouper
from wilds.common.utils import split_into_groups
from torch.autograd import grad

class ToxicClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embeddings_vectors, hidden_dim = 32, output_dim = 1):
        super(ToxicClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embeddings_vectors, freeze=True)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        _, embedded = self.rnn(embedded)
        return self.output(embedded[-1])

def tokenize(text, max_length = 100):
    tokenizer = get_tokenizer('basic_english')
    text = text.lower()
    text = re.sub(r"([.!?,'*])", r"", text)
    text = re.sub(r"([-])", r" ", text)
    tokens = tokenizer(text)
    if len(tokens) < max_length:
      tokens.extend(['<PAD>']*(max_length - len(tokens)))
    tokens = tokens[:max_length]
    tokens = [glove.stoi.get(token, len(glove.stoi) - 1) for token in tokens]
    tokens = np.array(tokens, dtype=np.int64)
    return tokens

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
glove = GloVe(name='6B', dim=token_dim)
padding_vector = torch.zeros(token_dim)
padding_token = '<PAD>'
glove.itos.append(padding_token)  
glove.stoi[padding_token] = len(glove.itos) - 1 
glove.vectors = torch.cat((glove.vectors, padding_vector.unsqueeze(0)), dim=0) 
batch_size = 32
token_dim = 50
length = 100

dataset = get_dataset(dataset="civilcomments", download=True)
train_data = dataset.get_subset(
    "train")
train_loader = get_train_loader("standard", train_data, batch_size=32)
test_data = dataset.get_subset(
    "val")
test_loader = get_train_loader("standard", train_data, batch_size=32)

from wilds.common.grouper import CombinatorialGrouper
unawareness = CombinatorialGrouper(dataset, ['identity_any'])

train_loader = get_train_loader(
    "group", train_data, grouper=unawareness, n_groups_per_batch=1, batch_size=32
)

negative_samples = (train_data.y_array == 0).sum()
positive_samples = (train_data.y_array == 1).sum()
pos_weight = negative_samples / positive_samples

model = ToxicClassifier(len(glove), token_dim, glove.vectors.to(device))
model = model.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
optimizer = optim.Adam(model.parameters(), lr = 1e-1)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience = 10)
num_epochs = 20
for epoch in range(0, num_epochs):
  train_loss = 0.0
  train_correct = 0
  train_total = 0
  model.train()
  pbar = tqdm(train_loader)
  for _ , cur_train in enumerate(pbar):
      input, label, metadata = cur_train
      if metadata[:,8].sum() == 0:
        input = tuple(map(tokenize, input))
        input = torch.Tensor(input).long().to(device)
        label = label.to(device)
        output = model(input).reshape(-1)
        loss = criterion(output, label.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        predicted_labels = (torch.sigmoid(output) >= 0.5).float()
        train_correct += (predicted_labels == label).sum().item()
        train_total += len(label)
        pbar.set_postfix(MSE=loss.item())
        train_loss += loss
  train_loss /= len(train_loader.dataset)
  scheduler.step(train_loss)
  train_acc = train_correct / train_total
  print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")  

  test_loss = 0.0
  test_correct = 0
  test_total = 0
  pbar = tqdm(test_loader)
  model.eval()
  with torch.no_grad():
      for i, data in enumerate(pbar, 0):
          input, label, groupings = cur_train
          input = tuple(map(tokenize, input))
          input = torch.Tensor(input).long().to(device)
          label = label.to(device)
          output = model(input).reshape(-1)
          loss = criterion(output, label.float())
          predicted_labels = (torch.sigmoid(output) >= 0.5).float()
          test_total += len(label)
          test_correct += (predicted_labels == label).sum().item()
          test_loss += loss
          pbar.set_postfix(MSE=loss.item())
      
  test_loss /= len(test_loader.dataset)
  test_acc = test_correct / test_total
  print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")


from wilds.common.data_loaders import get_eval_loader

# Get the test set
test_data = dataset.get_subset(
    "test",
)

# Prepare the data loader
test_loader = get_eval_loader("standard", test_data, batch_size=32)
trues = []
preds = []
metadatas = []
for input, true, metadata in test_loader:
    with torch.no_grad():
      input = tuple(map(tokenize, input))
      input = torch.Tensor(input).long().to(device)
      output = model(input)
      output = (torch.sigmoid(output) >= 0.5).long().reshape(-1)
      trues.append(true.to('cpu'))
      preds.append(output.to('cpu'))
      metadatas.append(metadata.to('cpu'))
all_preds = torch.cat(preds, dim = 0)
all_trues = torch.cat(trues, dim = 0)
all_metas = torch.cat(metadatas, dim = 0)
print(dataset.eval(all_preds, all_trues, all_metas))

  0%|          | 0/8407 [00:00<?, ?it/s]<ipython-input-10-443b30f9368f>:97: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  input = torch.Tensor(input).long().to(device)
100%|██████████| 8407/8407 [00:53<00:00, 158.59it/s, MSE=0.828]


Epoch 1/20, Train Loss: 0.0181, Train Acc: 0.6767


100%|██████████| 8408/8408 [00:49<00:00, 169.70it/s, MSE=1.06]


Epoch 1/20, Test Loss: 0.0331, Test Acc: 0.7812


100%|██████████| 8407/8407 [00:50<00:00, 167.12it/s, MSE=0.673]


...

Epoch 19/20, Test Loss: 0.0723, Test Acc: 0.7812


100%|██████████| 8407/8407 [00:51<00:00, 164.24it/s, MSE=0.836]


Epoch 20/20, Train Loss: 0.0188, Train Acc: 0.6688


100%|██████████| 8408/8408 [00:46<00:00, 181.49it/s, MSE=0.528]


Epoch 20/20, Test Loss: 0.0165, Test Acc: 0.6562
({'acc_avg': 0.6861236691474915, 'acc_y:0_male:1': 0.7440456748008728, 'count_y:0_male:1': 12092.0, 'acc_y:1_male:1': 0.24194280803203583, 'count_y:1_male:1': 2203.0, 'acc_y:0_female:1': 0.7530150413513184, 'count_y:0_female:1': 14179.0, 'acc_y:1_female:1': 0.23127754032611847, 'count_y:1_female:1': 2270.0, 'acc_y:0_LGBTQ:1': 0.7489096522331238, 'count_y:0_LGBTQ:1': 3210.0, 'acc_y:1_LGBTQ:1': 0.2565789520740509, 'count_y:1_LGBTQ:1': 1216.0, 'acc_y:0_christian:1': 0.7276257872581482, 'count_y:0_christian:1': 12101.0, 'acc_y:1_christian:1': 0.27222222089767456, 'count_y:1_christian:1': 1260.0, 'acc_y:0_muslim:1': 0.7521942257881165, 'count_y:0_muslim:1': 5355.0, 'acc_y:1_muslim:1': 0.258143812417984, 'count_y:1_muslim:1': 1627.0, 'acc_y:0_other_religions:1': 0.7442952990531921, 'count_y:0_other_religions:1': 2980.0, 'acc_y:1_other_religions:1': 0.26730769872665405, 'count_y:1_other_religions:1': 520.0, 'acc_y:0_black:1': 0.7526236772537231